In [92]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.cluster import KMeans

In [93]:
class RBFnet(nn.Module):
    def __init__(self, clusters):
        super(RBFnet, self).__init__()
        # remember how many centers we have
        self.N = clusters.shape[0]
        # our mean and sigmas for the RBF layer
        self.sigs = nn.Parameter(torch.ones(self.N,dtype=torch.float64)*5, requires_grad=False ) # our sigmas
        self.mus = nn.Parameter(torch.from_numpy(clusters), requires_grad=False ) # our means
        
        self.linear = nn.Linear(self.N, 1, dtype=torch.float64)

    def forward(self, x):
        distances = torch.sqrt(((x.unsqueeze(1) - self.mus)**2).sum(dim=2))
        # Calculate the Gaussian activations
        res = torch.exp((-0.5) * (distances**2) / self.sigs**2)
        # Set any NaN values to 0 (in case self.sigs is zero)
        res[res != res] = 0.0
        
        out = self.linear(res)
        return out

In [94]:
# Parameters
hidden_neurons = 20  # Use same as in training
model_path = "trained_rbfn.pth"
csv_input_path = "../hansa_test.csv"
csv_output_path = "predicted_output.csv"

In [95]:

# 1. Load input CSV
df = pd.read_csv(csv_input_path)
input_tensor = torch.tensor(df.values, dtype=torch.float32)  # shape: (N, 3)


In [96]:
# Cluster the data using KMeans with k=10
kmeans = KMeans(n_clusters=20)
kmeans.fit(input_tensor.cpu().numpy())
# find the cluster centers
clusters = kmeans.cluster_centers_.astype(float)
print(clusters.shape)

(20, 3)


In [97]:

# 2. Load model and weights
model = RBFnet(clusters)
model.load_state_dict(torch.load(model_path))
model.eval()

RBFnet(
  (linear): Linear(in_features=20, out_features=1, bias=True)
)

In [98]:
# 3. Predict
with torch.no_grad():
    predictions = model(input_tensor)  # shape: (N, 3)

In [99]:
# 4. Save predictions to CSV
pred_array = predictions.numpy()
df_pred = pd.DataFrame(pred_array, columns=["Cd"])
df_combined = pd.concat([df, df_pred], axis=1)

df_combined.to_csv(csv_output_path, index=False)
print(f"Predictions saved to {csv_output_path}")

Predictions saved to predicted_output.csv
